In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from typing import List, Union, Optional, Callable
import pickle
from Bio import AlignIO, SeqIO
from ete3 import Tree, TreeNode
from gctree import CollapsedTree
import nltk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
from ete3 import Tree, faces, TreeStyle, NodeStyle, TextFace, SequenceFace, COLOR_SCHEMES, CircleFace
from GCTree_preparation import *
import warnings
import math
warnings.filterwarnings("ignore")

path_to_storage = "/media/hieunguyen/HNHD01/storage/all_BSimons_datasets"
outdir = "/media/hieunguyen/GSHD_HN01/outdir/sc_bulk_BCR_data_analysis_v0.1"

inputdir = os.path.join(f"{outdir}/GEX_output/11_output")

all_input_files = [item for item in pathlib.Path(inputdir).glob("*.csv")]

for path_to_input_dataset in tqdm(all_input_files):
    sc_PROJECT = path_to_input_dataset.name.replace(".csv", "")
    path_to_05_output = os.path.join(outdir, "GEX_output", "05_output", sc_PROJECT)
    os.system(f"mkdir -p {path_to_05_output}")

    scdf = pd.read_csv(path_to_input_dataset, index_col = [0])
    scdf["CDR3_len"] = scdf["nSeqCDR3"].apply(lambda x: len(x))

    public_clonedf = pd.read_csv("public_clones.csv")

    match_public_clonedf = pd.DataFrame()
    for i in range(public_clonedf.shape[0]):
        v_gene = public_clonedf.iloc[i]["V_gene"]
        j_gene = public_clonedf.iloc[i]["J_gene"]
        public_sequence = public_clonedf.iloc[i]["aaseq"]
        tmpdf = scdf[(scdf["V.gene"] == v_gene) & (scdf["J.gene"] == j_gene)]
        tmpdf["public_CDR3nt"] = public_sequence
        match_public_clonedf = pd.concat([match_public_clonedf, tmpdf], axis = 0)

    match_public_clonedf["truncated_aaSeqCDR3"] = match_public_clonedf["aaSeqCDR3"].apply(lambda x: x[1:-1])
    match_public_clonedf["num_diff_public_clone"] = match_public_clonedf[["truncated_aaSeqCDR3", "public_CDR3nt"]].apply(lambda x: nltk.edit_distance(x[0], x[1]), axis = 1)
    match_public_clonedf["dist_to_public_clone"] = match_public_clonedf[["truncated_aaSeqCDR3", "public_CDR3nt"]].apply(lambda x: nltk.edit_distance(x[0], x[1])/min(len(x[0]), len(x[1])), axis = 1)

    match_public_clonedf["SampleID"] = match_public_clonedf["barcode"].apply(lambda x: "_".join(x.split("_")[0:-1]))

    match_public_clonedf.to_excel(os.path.join(path_to_05_output, f"{sc_PROJECT}_public_clone_full.xlsx"))
    match_public_clonedf[match_public_clonedf["dist_to_public_clone"] <= 0.15].\
        groupby("SampleID")["barcode"].\
        count().\
        reset_index().\
        sort_values(by = "barcode", ascending = False).to_excel(os.path.join(path_to_05_output, f"{sc_PROJECT}_public_clone_barcode_count.xlsx"))

    sig_match_public_clonedf = match_public_clonedf[match_public_clonedf["dist_to_public_clone"] <= 0.15]
    count_clonedf = pd.DataFrame(data = sig_match_public_clonedf["SampleID"].unique(), columns=["SampleID"])
    count_clonedf["numClone"] = count_clonedf["SampleID"].apply(lambda x: len(sig_match_public_clonedf[sig_match_public_clonedf['SampleID'] == x]["VJcombi_CDR3_0.85"].unique()))
    count_clonedf = count_clonedf.sort_values(by = "numClone", ascending = False)
    count_clonedf.to_excel(os.path.join(path_to_05_output, f"{sc_PROJECT}_public_unique_clone__count.xlsx"))